In [2]:
!pip install transformers
!pip install numpy
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import pipeline
import textwrap
import numpy as np
import pandas as pd
from pprint import pprint


In [4]:
df = pd.read_csv("Team_15_Dataset_Stock_Market_Analysis.csv")

In [5]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Titles
0,2019-03-13,184.449997,184.899994,180.399994,181.699997,181.135452,9769821.0,NaN
1,2019-03-14,181.500000,182.449997,177.649994,179.550003,178.992142,15737080.0,Geneva 2019: Tata Motors To Reveal New Generat...
2,2019-03-15,180.199997,181.399994,177.600006,180.000000,179.440735,17490993.0,"Not just Tata Motors, a lot of other companies..."
3,2019-03-17,NaN,NaN,NaN,NaN,NaN,NaN,Geneva 2019: Tata Motors To Reveal New Generat...
4,2019-03-18,181.000000,184.699997,179.500000,182.250000,181.683746,14690339.0,NaN


In [6]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [7]:
example = wrap(df['Titles'][1])
print(example)

Geneva 2019: Tata Motors To Reveal New Generation Passenger Vehicles.
Tata Harrier receives over 10000 bookings in India since launch.  Not
just Tata Motors, a lot of other companies are excited by Jaguar Land
Rover numbers.


In [8]:
summarizer = pipeline('summarization')      # Uses distilbart-cnn-12-6 model by default (Abstractive summarization technique)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [9]:
prompt = summarizer(wrap(df['Titles'][1]))[0]['summary_text']
print(prompt)

Your max_length is set to 142, but your input_length is only 52. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


 Tata Harrier receives over 10000 bookings in India since launch . Jaguar Land Rover receives over 10,000 bookings since launch in India . Tata Motors to unveil new generation of vehicles in Geneva, Geneva, Germany, Switzerland next month . Tata also set to unveil a new vehicle in Geneva next week .


In [10]:
# from transformers import pipeline
# classifier = pipeline(task="text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", top_k=None)

# sentences = [title for title in df['Titles'].fillna('')]

# # declare a new dataframe with labels positive, negative and neutral
# df = pd.DataFrame(columns=['Positive', 'Negative', 'Neutral'])

# for title in sentences[:5]:
#     model_outputs = classifier(title)
#     print(model_outputs[0])



In [11]:
df.head

<bound method NDFrame.head of             Date         Open         High          Low        Close  \
0     2019-03-13   184.449997   184.899994   180.399994   181.699997   
1     2019-03-14   181.500000   182.449997   177.649994   179.550003   
2     2019-03-15   180.199997   181.399994   177.600006   180.000000   
3     2019-03-17          NaN          NaN          NaN          NaN   
4     2019-03-18   181.000000   184.699997   179.500000   182.250000   
...          ...          ...          ...          ...          ...   
1633  2024-03-07  1025.000000  1047.000000  1014.049988  1039.300049   
1634  2024-03-09          NaN          NaN          NaN          NaN   
1635  2024-03-11  1034.750000  1035.949951  1021.000000  1028.000000   
1636  2024-03-12  1027.800049  1037.000000  1011.299988  1016.500000   
1637  2024-03-13  1019.000000  1021.000000   985.000000   995.299988   

        Adj Close      Volume  \
0      181.135452   9769821.0   
1      178.992142  15737080.0   
2     

In [30]:
import pandas as pd
from transformers import pipeline

# Load sentiment analysis pipeline
classifier = pipeline(task="text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", top_k=None)

# declare a new dataframe with labels positive, negative and neutral
new_df = pd.DataFrame()

# Function to classify sentiment and update the DataFrame
def classify_sentiment(title):
  model_outputs = classifier(title)
  if not model_outputs:
      return pd.Series({'Positive': 0, 'Negative': 0, 'Neutral': 1})
  
  sentiment_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
  sorted_outputs = sorted(model_outputs[0], key=lambda x: sentiment_mapping[x['label']], reverse=True)

# Return a Series with the scores
  return pd.Series({
    'Neutral': sorted_outputs[0]['score'] if sorted_outputs[0]['label'] == 'neutral' else 0,
    'Negative': sorted_outputs[1]['score'] if sorted_outputs[1]['label'] == 'negative' else 0,
    'Positive': sorted_outputs[2]['score'] if sorted_outputs[2]['label'] == 'positive' else 0
})


# # Classify sentiment for each title and update the new DataFrame
new_df = pd.concat([new_df, df['Titles'].fillna('').apply(classify_sentiment)], axis=1)


# # Display the new DataFrame
print(new_df)


       Neutral  Negative  Positive
0     0.999858  0.000085  0.000058
1     0.001159  0.000286  0.998555
2     0.001232  0.000302  0.998467
3     0.999858  0.000053  0.000089
4     0.999858  0.000085  0.000058
...        ...       ...       ...
1633  0.999858  0.000085  0.000058
1634  0.000644  0.000256  0.999100
1635  0.999858  0.000085  0.000058
1636  0.999858  0.000085  0.000058
1637  0.999858  0.000085  0.000058

[1638 rows x 3 columns]


In [41]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Titles
0,2019-03-13,184.449997,184.899994,180.399994,181.699997,181.135452,9769821.0,NaN
1,2019-03-14,181.500000,182.449997,177.649994,179.550003,178.992142,15737080.0,Geneva 2019: Tata Motors To Reveal New Generat...
2,2019-03-15,180.199997,181.399994,177.600006,180.000000,179.440735,17490993.0,"Not just Tata Motors, a lot of other companies..."
3,2019-03-17,NaN,NaN,NaN,NaN,NaN,NaN,Geneva 2019: Tata Motors To Reveal New Generat...
4,2019-03-18,181.000000,184.699997,179.500000,182.250000,181.683746,14690339.0,NaN


In [40]:
new_df.head()

,Neutral,Negative,Positive
0,0.999858,0.000085,0.000058
1,0.001159,0.000286,0.998555
2,0.001232,0.000302,0.998467
3,0.999858,0.000053,0.000089
4,0.999858,0.000085,0.000058


In [42]:
merged_df = pd.concat([df, new_df], axis=1)

merged_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Titles,Neutral,Negative,Positive
0,2019-03-13,184.449997,184.899994,180.399994,181.699997,181.135452,9769821.0,NaN,0.999858,0.000085,0.000058
1,2019-03-14,181.500000,182.449997,177.649994,179.550003,178.992142,15737080.0,Geneva 2019: Tata Motors To Reveal New Generat...,0.001159,0.000286,0.998555
2,2019-03-15,180.199997,181.399994,177.600006,180.000000,179.440735,17490993.0,"Not just Tata Motors, a lot of other companies...",0.001232,0.000302,0.998467
3,2019-03-17,NaN,NaN,NaN,NaN,NaN,NaN,Geneva 2019: Tata Motors To Reveal New Generat...,0.999858,0.000053,0.000089
4,2019-03-18,181.000000,184.699997,179.500000,182.250000,181.683746,14690339.0,NaN,0.999858,0.000085,0.000058


In [43]:
merged_df.to_csv('merged_data.csv', index=False)